In [ ]:
import os
import openai
import json
import numpy as np
import pandas as pd
from datasets import load_dataset as ld
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import time

In [ ]:
openai.api_key = "Enter OPENAI Key"

In [ ]:
from datasets import load_dataset as ld

def load_dataset_as_pandas_dataframe(
    dataset_name: str="trec"
):

    dataset = ld(dataset_name)
    df = dataset["train"].to_pandas()
    df = df.sample(frac=1)

    df_test = dataset["test"].to_pandas()
    df_test = df_test.sample(frac=1)
    
    return df, df_test

In [ ]:
df, df_test = load_dataset_as_pandas_dataframe()

In [ ]:
df['fine_label'].unique()

In [ ]:
label_to_id = {
    'Abbreviation': 0,
    'Expression abbreviated': 1,
    'Animal': 2,
    'Organ of body': 3,
    'Color': 4,
    'Invention, book and other creative piece': 5,
    'Currency name': 6,
    'Disease and medicine': 7,
    'Event': 8,
    'Food': 9,
    'Musical instrument': 10,
    'Language': 11,
    'Letter like a-z': 12,
    'Other entity': 13,
    'Plant': 14,
    'Product': 15,
    'Religion': 16,
    'Sport': 17,
    'Element and substance': 18,
    'Symbols and sign': 19,
    'Techniques and method': 20,
    'Equivalent term': 21,
    'Vehicle': 22,
    'Word with a special property': 23,
    'Definition of something': 24,
    'Description of something': 25,
    'Manner of an action': 26,
    'Reason': 27,
    'Group or organization of persons': 28,
    'Individual': 29,
    'Title of a person': 30,
    'Description of a person': 31,
    'City': 32,
    'Country': 33,
    'Mountain': 34,
    'Other location': 35,
    'State': 36,
    'Postcode or other code': 37,
    'Number of something': 38,
    'Date': 39,
    'Distance, linear measure': 40,
    'Price': 41,
    'Order, rank': 42,
    'Other number': 43,
    'Lasting time of something': 44,
    'Percent, fraction': 45,
    'Speed': 46,
    'Temperature': 47,
    'Size, area and volume': 48,
    'Weight': 49
}



id_to_label = {value : key for key, value in label_to_id.items()} 


In [ ]:

categories = list(label_to_id.keys())

# DataFrame mapping

In [ ]:
df["fine_label"] = df["fine_label"].map(id_to_label)
df_test["fine_label"] = df_test["fine_label"].map(id_to_label)

In [ ]:
null_values = df.isnull().sum()
print(null_values)

# Let create a text_to_openai_json function

In [ ]:
def text_to_openai_json(data,filename):
    """
    Converts a given dataset into a JSON Lines (JSONL) file suitable for OpenAI's GPT-3.5 turbo model.
    
    Args:
        data (DataFrame or similar data structure): Input data containing text and labels.

    The function processes the input data row by row, constructing conversations for each row with a system message, user message, and an assistant message. It then writes the generated conversation data to a JSONL file.
 
    """
    # Initialize an empty list to store conversation data
    message_list = []

    # Iterate through the rows in the input data
    for _, row in data.iterrows():
        # Create a system message as an initial instruction
        system_message = {
            "role": "system",
            "content": f"given the following text: find the category in: {categories} that is most closely associated with it. Return only the category name"
        }

        # Append the system message to the conversation
        message_list.append({"messages": [system_message]})

        # Create a user message based on the 'text' column from the data
        user_message = {
            "role": "user",
            "content": row['text']
        }

        # Append the user message to the conversation
        message_list[-1]["messages"].append(user_message)

        # Create an assistant message based on the 'fine_label' column from the data
        assistant_message = {
            "role": 'assistant',
            "content": row['fine_label']
        }

        # Append the assistant message to the conversation
        message_list[-1]["messages"].append(assistant_message)

    # Write the conversation data to a JSON Lines (JSONL) file
    with open(filename, "w") as json_file:
        for message in message_list:
            # Serialize the conversation data to JSON and write it to the file
            json.dump(message, json_file)
            json_file.write("\n")




# Few Shot Prompting


In [ ]:
system_content =  f"given the following text: find the category in: {categories} that is most closely associated with it. Return only the category name only in following format"

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
def zero_shot_model(data,model_id):
    pred = []
    for row in data["text"]:
        completion = openai.ChatCompletion.create(
            model= model_id ,
            messages=[
                {"role": "system", "content": system_content },
                {"role": "user", "content": row }
            ])
        
        print(f'text: {row}')
        print(completion.choices[0].message.content)
        pred.append(completion.choices[0].message.content)
    pred_df = pd.DataFrame({'text': data["text"], 'fine_label' : data['fine_label'], 'few-shot predictions' : pred })
    

    return pred_df

In [ ]:
cumulative_increment = 0
model_id = 'gpt-3.5-turbo-0301'
all_model_id = []

In [ ]:
def fine_tune_model(model_id,num_label,pred_df):
    incorrection_pred_df = pred_df[pred_df['fine_label'] != pred_df['few-shot predictions']][:num_label]
    filename = f'ft_increment_{num_label}.jsonl'
    text_to_openai_json(incorrection_pred_df, filename)
    loader = openai.File.create(file=open(filename, "rb"), purpose='fine-tune')
    fine_tuning_job = openai.FineTuningJob.create(training_file=loader.id, model="gpt-3.5-turbo")
    return fine_tuning_job.id

In [ ]:
def wait_for_fine_tuning(job_id):
    while True:
        response = openai.FineTuningJob.retrieve(job_id)
        print(response["fine_tuned_model"])
        if response["fine_tuned_model"]:
            print(response["fine_tuned_model"])
            return response["fine_tuned_model"]
        time.sleep(60)  # Check every 60 seconds

In [ ]:
def ft_accuracy(data,model_id):
    pred = []
    for row in data["text"]:
        completion = openai.ChatCompletion.create(
            model= model_id ,
            messages=[
                {"role": "system", "content": system_content },
                {"role": "user", "content": row }
            ])
        
        print(f'text: {row}')
        print(completion.choices[0].message.content)
        pred.append(completion.choices[0].message.content)
        
    accuracy = accuracy_score(data['fine_label'], pred)
    print(f'Accuracy: {accuracy * 100:.2f}%')
    precision, recall, f1, _ = precision_recall_fscore_support(data['fine_label'], pred, average='macro',zero_division=1)
    
    return accuracy, precision, recall, f1, pred



# Zero Shot Model

In [ ]:
pred_df = zero_shot_model(model_id='gpt-3.5-turbo-0301', data = df_test)

In [ ]:
pred_df.head()

In [ ]:
accuracy_score(pred_df['fine_label'].to_list() ,pred_df['few-shot predictions'].to_list()) 

# Fine-Tuning GPT-3.5 Turbo on Iterative Label Increase

In [ ]:
model_ids = []
accs = [] 
precisions = [] 
recalls = [] 
f1s = []
label_count = 0
labels = []
for i in range(15):
    label_count += 10
    labels.append(label_count)
    ft_id = fine_tune_model(model_id = 'gpt-3.5-turbo-0301', num_label=label_count, pred_df=pred_df)
    if wait_for_fine_tuning(ft_id) is not None:
        model_ids.append(wait_for_fine_tuning(ft_id))
        accuracy, precision, recall, f1, pred = ft_accuracy(data=df_test,model_id=(wait_for_fine_tuning(ft_id)))
        accs.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1s.append(f1)
        

In [ ]:
data = {
    'Label Count': labels,
    'Accuracy': accs,
    'Precision': precisions,
    'Recall': recalls,
    'F1 Score': f1s
}

df_results_acc= pd.DataFrame(data)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(12, 8))
ax = plt.axes()
ax.plot(labels, accs, label='GPT3.5 Turbo')
ax.legend()
plt.ylabel('accuracy')
plt.xlabel('labels')

In [ ]:
df_results_acc.to_csv('FT_GPT3.5_fine_label_metrics_results.csv',index=False)

In [ ]:
df_test.shape0

In [ ]:
data = {'pred': pred, 'fine_label':df_test['fine_label']}

In [ ]:
df_predictions = pd.DataFrame(data)

In [ ]:
df_predictions.to_csv('FT_150_GPT3.5_PRED.csv',index=False)

In [ ]:
df_predictions[df_predictions['pred']!= df_predictions['fine_label']].shape